#### Introduction

In this notebook, we are going to discuss about the model registry. We have two models - xgboost and randomforest. We need to register the model to mlflow and from there we can migrate the models to production/ staging/ archive.

We can do this using the mlflow ui as well, but for automation, it is better to use code. This will also be useful while deploying the best model in next iterations. For this we will use the mlflow client class. We can do all the things that we can do using the UI. Basically, the client  is required to interact with the mlflow server. Let's start

In [1]:
from mlflow.tracking import MlflowClient

##### Initiate an instance of the client

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri= MLFLOW_TRACKING_URI)

Let's check the list of experiments

In [3]:
client.search_experiments()

[<Experiment: artifact_location='./mlruns/1', creation_time=1673260171550, experiment_id='1', last_update_time=1673260171550, lifecycle_stage='active', name='persistency-prediction-experiment', tags={'mlflow.note.content': 'This experiment is for running different trials for '
                         'predicting whether a customer will pay the renewal '
                         'premium for 1st year of a term life insurance.'}>]

##### Get the best result runs

Now, we want to find out the best results based on some filter criteria (highest recall) and also tags

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= '1',
    filter_string= "tags.type = 'xgboost-sklearn final'",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results= 5,
    order_by= ["metrics.recall DESC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, recall: {run.data.metrics['recall']:.4f}")

run id: 51f002c6c39e4baaae119815cb262aca, recall: 0.8137


We know that `run id: b878a2ba0b834edea0a44cf6935f4dc0` is for the **xgboost** model and `run id: 0a64c24edd994328b2a0e2749272b48c` is for the **randomforest** model.
Let's now register the models


In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

#### For xgboost model

In [9]:
run_id = run.info.run_id
MODEL_URI = f"runs:/{run_id}/model"

mlflow.register_model(model_uri= MODEL_URI, name = 'persistency-prediction-classifier')

Registered model 'persistency-prediction-classifier' already exists. Creating a new version of this model...
2023/01/26 17:40:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: persistency-prediction-classifier, version 4
Created version '4' of model 'persistency-prediction-classifier'.


<ModelVersion: creation_timestamp=1674735054453, current_stage='None', description=None, last_updated_timestamp=1674735054453, name='persistency-prediction-classifier', run_id='51f002c6c39e4baaae119815cb262aca', run_link=None, source='./mlruns/1/51f002c6c39e4baaae119815cb262aca/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

#### For Randomforest Model

In [59]:
run_id = '0a64c24edd994328b2a0e2749272b48c'
MODEL_URI = f"runs:/{run_id}/model"

mlflow.register_model(model_uri= MODEL_URI, name = 'persistency-prediction-classifier')

Registered model 'persistency-prediction-classifier' already exists. Creating a new version of this model...
2023/01/11 22:57:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: persistency-prediction-classifier, version 3
Created version '3' of model 'persistency-prediction-classifier'.


#### Let's check the latest versions of the registered model

In [12]:
MODEL_NAME = 'persistency-prediction-classifier'
latest_versions = client.get_latest_versions(name = MODEL_NAME, stages= ["None"])

for version in latest_versions:
    print(f"name: {version.name}")
    print(f"latest version: {version.version}")
    print(f"run_id: {version.run_id}")
    print(f"current_stage: {version.current_stage}")

name: persistency-prediction-classifier
latest version: 4
run_id: 51f002c6c39e4baaae119815cb262aca
current_stage: None


#### Moving xgboost model `version 1` to `Production`

In [50]:
MODEL_VERSION = 1
NEW_STAGE = "Production"
client.transition_model_version_stage(name = MODEL_NAME,
                                    version = MODEL_VERSION,
                                    stage= NEW_STAGE,
                                    archive_existing_versions= False)

<ModelVersion: creation_timestamp=1673454908705, current_stage='Production', description=None, last_updated_timestamp=1673457117476, name='persistency-prediction-classifier', run_id='b878a2ba0b834edea0a44cf6935f4dc0', run_link=None, source='./mlruns/1/b878a2ba0b834edea0a44cf6935f4dc0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

#### Moving xgboost model `version 2` to `Staging`

In [51]:
MODEL_VERSION = 2
NEW_STAGE = "Staging"

client.transition_model_version_stage(name = MODEL_NAME,
                                    version = MODEL_VERSION,
                                    stage= NEW_STAGE,
                                    archive_existing_versions= False)

<ModelVersion: creation_timestamp=1673456026710, current_stage='Staging', description=None, last_updated_timestamp=1673457150555, name='persistency-prediction-classifier', run_id='0a64c24edd994328b2a0e2749272b48c', run_link=None, source='./mlruns/1/0a64c24edd994328b2a0e2749272b48c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

#### Adding descrptions to the model versions

In [57]:
from datetime import datetime
MODEL_VERSION = 1
CHANGE_DATE = datetime.today().date()

client.update_model_version(
    name = MODEL_NAME,
    version= MODEL_VERSION,
    description= f"This is the xgboost model with recall 0.832 and this model version {version} was transitioned to Production on {CHANGE_DATE}"
)

<ModelVersion: creation_timestamp=1673454908705, current_stage='Production', description=('This is the xgboost model with recall 0.832 and this model version '
 "<ModelVersion: creation_timestamp=1673456026710, current_stage='None', "
 'description=None, last_updated_timestamp=1673456026710, '
 "name='persistency-prediction-classifier', "
 "run_id='0a64c24edd994328b2a0e2749272b48c', run_link=None, "
 "source='./mlruns/1/0a64c24edd994328b2a0e2749272b48c/artifacts/model', "
 "status='READY', status_message=None, tags={}, user_id=None, version=2> was "
 'transitioned to Production on 2023-01-11'), last_updated_timestamp=1673457601648, name='persistency-prediction-classifier', run_id='b878a2ba0b834edea0a44cf6935f4dc0', run_link=None, source='./mlruns/1/b878a2ba0b834edea0a44cf6935f4dc0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [64]:
MODEL_VERSION = 2
CHANGE_DATE = datetime.today().date()

client.update_model_version(
    name = MODEL_NAME,
    version= MODEL_VERSION,
    description= f"This is the randomforest model with recall 0.809 and this model version {version} was transitioned to Stagiing on {CHANGE_DATE}"
)

<ModelVersion: creation_timestamp=1673456026710, current_stage='Staging', description=('This is the randomforest model with recall 0.809 and this model version '
 "<ModelVersion: creation_timestamp=1673456026710, current_stage='None', "
 'description=None, last_updated_timestamp=1673456026710, '
 "name='persistency-prediction-classifier', "
 "run_id='0a64c24edd994328b2a0e2749272b48c', run_link=None, "
 "source='./mlruns/1/0a64c24edd994328b2a0e2749272b48c/artifacts/model', "
 "status='READY', status_message=None, tags={}, user_id=None, version=2> was "
 'transitioned to Stagiing on 2023-01-11'), last_updated_timestamp=1673458485106, name='persistency-prediction-classifier', run_id='0a64c24edd994328b2a0e2749272b48c', run_link=None, source='./mlruns/1/0a64c24edd994328b2a0e2749272b48c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

#### Moving xgboost model `version 4` to `Production`

In [17]:
for version in latest_versions:
    LATEST_VERSION = version.version
    print(LATEST_VERSION)


4


In [18]:
for version in latest_versions:
    LATEST_VERSION = version.version

MODEL_VERSION = LATEST_VERSION
NEW_STAGE = "Production"
client.transition_model_version_stage(name = MODEL_NAME,
                                    version = MODEL_VERSION,
                                    stage= NEW_STAGE,
                                    archive_existing_versions= True)

<ModelVersion: creation_timestamp=1674735054453, current_stage='Production', description=None, last_updated_timestamp=1674735428967, name='persistency-prediction-classifier', run_id='51f002c6c39e4baaae119815cb262aca', run_link=None, source='./mlruns/1/51f002c6c39e4baaae119815cb262aca/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [20]:
from datetime import datetime

In [21]:
MODEL_VERSION = LATEST_VERSION
CHANGE_DATE = datetime.today().date()

client.update_model_version(
    name = MODEL_NAME,
    version= MODEL_VERSION,
    description= f"This is the xgboost model with scikit learn wrapper with recall 0.8137 and this model version {version} was transitioned to Production on {CHANGE_DATE}"
)

<ModelVersion: creation_timestamp=1674735054453, current_stage='Production', description=('This is the xgboost model with scikit learn wrapper with recall 0.8137 and '
 'this model version <ModelVersion: creation_timestamp=1674735054453, '
 "current_stage='None', description=None, "
 'last_updated_timestamp=1674735054453, '
 "name='persistency-prediction-classifier', "
 "run_id='51f002c6c39e4baaae119815cb262aca', run_link=None, "
 "source='./mlruns/1/51f002c6c39e4baaae119815cb262aca/artifacts/model', "
 "status='READY', status_message=None, tags={}, user_id=None, version=4> was "
 'transitioned to Production on 2023-01-26'), last_updated_timestamp=1674736031641, name='persistency-prediction-classifier', run_id='51f002c6c39e4baaae119815cb262aca', run_link=None, source='./mlruns/1/51f002c6c39e4baaae119815cb262aca/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>